In [1]:
from mosaic_dataset import MosaicDataModule
import json
from pathlib import Path

/data/training/pypoetry/virtualenvs/multipage-classifier-xptqA-4L-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:

DATASET_PATH = "/data/training/master_thesis/datasets/2023-05-23"
CLASS_PATH = "/data/training/master_thesis/datasets/bzuf_classes.json"
BATCH_SIZE = 1
MAX_PAGES = 4
NUM_WORKERS = 0

In [3]:
from multipage_classifier.encoder.swin_encoder import SwinEncoder, SwinEncoderConfig

PRETRAINED_MODEL = "microsoft/swinv2-small-patch4-window8-256"
IMAGE_SIZE = (704, 512) # height, width TODO 1024, 1408
swin_config = SwinEncoderConfig(
        image_size=IMAGE_SIZE, 
        pretrained_model_name_or_path=PRETRAINED_MODEL
    )
model = SwinEncoder(swin_config)

Some weights of SwinModel were not initialized from the model checkpoint at microsoft/swinv2-small-patch4-window8-256 and are newly initialized: ['swin.encoder.layers.2.blocks.0.attention.self.value.weight', 'swin.encoder.layers.2.blocks.6.layernorm_after.weight', 'swin.encoder.layers.2.blocks.7.attention.self.key.bias', 'swin.encoder.layers.2.blocks.11.attention.self.relative_position_bias_table', 'swin.encoder.layers.2.blocks.5.attention.output.dense.weight', 'swin.encoder.layers.2.blocks.8.attention.self.relative_position_index', 'swin.encoder.layers.1.blocks.1.intermediate.dense.bias', 'swin.encoder.layers.3.blocks.1.attention.self.query.bias', 'swin.encoder.layers.3.blocks.0.attention.self.query.weight', 'swin.encoder.layers.3.blocks.0.layernorm_before.weight', 'swin.encoder.layers.2.blocks.14.attention.output.dense.weight', 'swin.encoder.layers.1.blocks.0.attention.self.key.weight', 'swin.encoder.layers.2.blocks.5.attention.output.dense.bias', 'swin.encoder.layers.2.blocks.11.att

In [4]:
classes = [c for c in json.load(open(CLASS_PATH))]

data_module = MosaicDataModule(Path(DATASET_PATH), classes, model.prepare_input, batch_size=BATCH_SIZE, max_pages=MAX_PAGES, num_workers=NUM_WORKERS)

data_module.prepare_data()
data_module.setup()

In [17]:
id2class = {idx: str(label) for idx, label in enumerate(classes)}

target = ["antrag.formlos", "antrag.formblattantrag.hilfe_zur_pflege", "vermoegen.vermoegenserklaerung"]
for idx, sample in enumerate(data_module.train_dataloader()):
    doc_classes = [id2class[cls.item()] for cls in sample["doc_class"]]
    if all(c in doc_classes for c in target):
        print(idx, doc_classes)

In [18]:
[c in classes for c in target]

[True, True, True]

In [6]:
batch["doc_id"]

tensor([0, 0, 1, 2])

In [7]:
from transformers import MBartConfig, MBartForCausalLM, MBartTokenizer

tokenizer: MBartTokenizer = MBartTokenizer.from_pretrained("facebook/mbart-large-en-ro")
tokenizer.add_special_tokens({"additional_special_tokens": sorted(set(["<sep/>"]))})

1

In [11]:

from typing import Any


def json2token(
    obj: Any,
    update_special_tokens_for_json_key: bool = True,
    sort_json_key: bool = True,
) -> str:
    """
    Convert an ordered JSON object into a token sequence
    """
    if type(obj) == dict:
        if len(obj) == 1 and "text_sequence" in obj:
            return obj["text_sequence"]
        else:
            output = ""
            if sort_json_key:
                keys = sorted(obj.keys(), reverse=True)
            else:
                keys = obj.keys()
            for k in keys:
                output += (
                    rf"<s_{k}>"
                    + json2token(
                        obj[k], update_special_tokens_for_json_key, sort_json_key
                    )
                    + rf"</s_{k}>"
                )
            return output
    elif type(obj) == list:
        print("§FDDFSDFS")
        return r"<sep/>".join(
            [
                json2token(
                    item, update_special_tokens_for_json_key, sort_json_key
                )
                for item in obj
            ]
        )
    else:
        obj = str(obj)
        if f"<{obj}/>" in tokenizer.all_special_tokens:
            obj = f"<{obj}/>"  # for categorical special tokens
        return obj


In [12]:
json2token([{"doc_id": 0, "doc_class": "abc"}, {"doc_id": 0, "doc_class": "abc"}])

§FDDFSDFS


'<s_doc_id>0</s_doc_id><s_doc_class>abc</s_doc_class><sep/><s_doc_id>0</s_doc_id><s_doc_class>abc</s_doc_class>'